In [1]:
!pip install trax
import numpy as np
import pandas as pd
from trax import layers as tl
from nltk.tokenize import word_tokenize
from trax.fastmath import numpy as np
import random as rnd
from trax.supervised.training import TrainTask,EvalTask,Loop
import trax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 9.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
beatrix-jupyterlab

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data=pd.read_csv(r'/kaggle/input/ner-dataset/ner_datasetreference.csv',encoding = "ISO-8859-1")

In [3]:
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
index=data.dropna().index
index

Int64Index([      0,      24,      54,      68,      83,     108,     132,
                153,     181,     196,
            ...
            1048375, 1048400, 1048422, 1048438, 1048472, 1048501, 1048521,
            1048545, 1048556, 1048567],
           dtype='int64', length=47959)

In [5]:
sentence=[]
tag=[]
for idx,i in enumerate(index):
    one_Sentence=''
    one_tag=''
    if i ==index[-1]:
        break
    for i in range(index[idx],index[idx+1]):
        one_Sentence+=' '+data.iloc[i]['Word']
        one_tag+=' '+data.iloc[i]['Tag']
    sentence.append(one_Sentence)
    tag.append(one_tag)

In [6]:
print(sentence[3])
print(tag[3])
data.iloc[index[3]:index[4]]

 Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .
 O O O O O O O O O O O O O O O


,Sentence #,Word,POS,Tag
68,Sentence: 4,Police,NNS,O
69,NaN,put,VBD,O
70,NaN,the,DT,O
71,NaN,number,NN,O
72,NaN,of,IN,O
73,NaN,marchers,NNS,O
74,NaN,at,IN,O
75,NaN,"10,000",CD,O
76,NaN,while,IN,O
77,NaN,organizers,NNS,O


In [7]:
print('data size=',len(sentence))
data_size=len(sentence)
train_data=sentence[:int(data_size*.8)]
test_data=sentence[len(train_data):int(data_size*.9)]
val_data=sentence[int(data_size*.9):]
train_label=tag[:int(data_size*.8)]
test_label=tag[len(train_data):int(data_size*.9)]
val_label=tag[int(data_size*.9):]
print(len(train_data),len(test_data),len(val_data))
print(len(train_label),len(test_label),len(val_label))
print(train_data[0])
print(train_label[0])

data size= 47958
38366 4796 4796
38366 4796 4796
 Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
 O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O


In [182]:
def word_to_number (data):
    vocab={'<pad>':0,'<unKnow>':1}
    for i in data:
        words=word_tokenize(i)
        for word in words:
            if word not in list(vocab):
                vocab[word]=len(vocab)
    return vocab


In [183]:
vocab_word=word_to_number(train_data)

In [184]:
vocab_word['<pad>']

0

In [287]:
def tags_to_number (label):
    vocab_tags={"<pad>":0}
    number_to_tags={0:'<pad>'}
    for tags in label:
        tags=tags.split()
        for tag in tags:
            if tag not in list(vocab_tags):
                vocab_tags[tag]=len(vocab_tags)
                number_to_tags[len(number_to_tags)]=tag
    return vocab_tags,number_to_tags


In [288]:
vocab_tags,number_to_tags=tags_to_number(train_label)

In [289]:
number_to_tags

{0: '<pad>',
 1: 'O',
 2: 'B-geo',
 3: 'B-gpe',
 4: 'B-per',
 5: 'I-geo',
 6: 'B-org',
 7: 'I-org',
 8: 'B-tim',
 9: 'B-art',
 10: 'I-art',
 11: 'I-per',
 12: 'I-gpe',
 13: 'I-tim',
 14: 'B-nat',
 15: 'B-eve',
 16: 'I-eve',
 17: 'I-nat'}

In [260]:
def tokenize_sentence(sentence,vocab):
    tokenize_sentence=[]
    words=word_tokenize(sentence)
    for word in words:
        
        tokenize_sentence.append(vocab.get(word,1))
    return tokenize_sentence
        

In [189]:
def gnerator_data(data,label):
    data_size=len(data)
    index=[*range(data_size)]
    idx=0
    while True:
        if idx>=data_size:
            idx=0
        rnd.shuffle(index)
        sentence=tokenize_sentence(data[index[idx]],vocab_word)
        tags=tokenize_sentence(label[index[idx]],vocab_tags)
        idx+=1
        X=np.array(sentence)
        Y=np.array(tags)
        yield ((X,Y))

In [208]:
def gnerator_test_data(data,label):
    data_size=len(data)
    index=[*range(data_size)]
    X=[]
    Y=[]
    while True:
        for i in index:
            sentence=tokenize_sentence(data[index[idx]],vocab_word)
            tags=tokenize_sentence(label[index[idx]],vocab_tags)
            Y.append(tags)
            X.append(sentence)
        X=np.array(X)
        Y=np.array(Y)
        yield ((X,Y))

In [190]:
X,Y=next(gnerator_data(train_data,train_label))
X.shape,Y.shape

((12,), (12,))

In [191]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [256,128, 64, 32, 16,    8,   4,  2]


# Create the generators.
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(train_data,train_label))

val_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(val_data,val_label))


test_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(test_data,test_label))

train_batch_stream = trax.data.AddLossWeights(id_to_mask=vocab_word['<pad>'])(train_batch_stream)
val_batch_stream = trax.data.AddLossWeights(id_to_mask=vocab_word['<pad>'])(val_batch_stream)


In [192]:
X,Y,M=next(train_batch_stream)
X.shape,Y.shape,M.shape

((64, 32), (64, 32), (64, 32))

In [193]:
X[0]

array([7710,  287,  174,   11, 1815,   70, 7711, 2922,   61, 5231,  193,
       7712,   47,  265,  282, 1317,   23,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [194]:
Y[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [195]:
M[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [196]:
def Model(d_model,n_layer):
    model=tl.Serial([tl.Embedding(len(vocab_word),d_model),
                         [tl.LSTM(d_model) for _ in range(n_layer)],
                          tl.Dense(len(vocab_tags)),
                          tl.LogSoftmax()

    ])
    return model

In [197]:
def train_model(train_batch_stream,val_batch_stream,n_steps,model,output_dir='./model'):
    train_task=TrainTask(train_batch_stream,
                         loss_layer=tl.CrossEntropyLoss(),
                         optimizer=trax.optimizers.Adam(0.001),
                         n_steps_per_checkpoint=100)
    
    eval_task=EvalTask(val_batch_stream,
                        metrics=[tl.CrossEntropyLoss(),tl.Accuracy()],
                      )
    
    training_loop =Loop(
        model, # A model to train
        train_task, # A train task
        eval_tasks = [eval_task], # The evaluation task
        output_dir = output_dir # The output directory
    )

    # Train with train_steps
    training_loop.run(n_steps = n_steps)
### END CODE HERE ###
    return training_loop
    


In [200]:
model=Model(512,3)
n_steps=int((len(train_data)/64))*3
print(n_steps)

training=train_model(train_batch_stream,val_batch_stream,n_steps,model,output_dir='./model')

1797


/opt/conda/lib/python3.10/site-packages/trax/layers/base.py:851: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip.GzipFile(fileobj=f, compresslevel=compresslevel) as gzipf:



Step      1: Total number of trainable weights: 22593554
Step      1: Ran 1 train steps in 17.03 secs
Step      1: train CrossEntropyLoss |  2.75794077


/opt/conda/lib/python3.10/site-packages/trax/supervised/training.py:1249: FutureWarning: GzipFile was opened for writing, but this will change in future Python releases.  Specify the mode argument for opening it for writing.
  with gzip_lib.GzipFile(fileobj=f, compresslevel=2) as gzipf:


Step      1: eval  CrossEntropyLoss |  1.21168351
Step      1: eval          Accuracy |  0.82526177

Step    100: Ran 99 train steps in 423.11 secs
Step    100: train CrossEntropyLoss |  0.77143353
Step    100: eval  CrossEntropyLoss |  0.78344017
Step    100: eval          Accuracy |  0.83703703

Step    200: Ran 100 train steps in 420.96 secs
Step    200: train CrossEntropyLoss |  0.75090826
Step    200: eval  CrossEntropyLoss |  0.83843350
Step    200: eval          Accuracy |  0.82199830

Step    300: Ran 100 train steps in 425.85 secs
Step    300: train CrossEntropyLoss |  0.72933584
Step    300: eval  CrossEntropyLoss |  0.70461553
Step    300: eval          Accuracy |  0.85246950

Step    400: Ran 100 train steps in 424.21 secs
Step    400: train CrossEntropyLoss |  0.71576393
Step    400: eval  CrossEntropyLoss |  0.62360048
Step    400: eval          Accuracy |  0.87216735

Step    500: Ran 100 train steps in 420.69 secs
Step    500: train CrossEntropyLoss |  0.70304674
Step  

In [216]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [len(test_data)]*7


test_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(gnerator_data(test_data,test_label))

train_batch_stream = trax.data.AddLossWeights(id_to_mask=vocab_word['<pad>'])(test_batch_stream)


In [217]:
X,Y,M=next(train_batch_stream)
X.shape,Y.shape,M.shape

((4796, 32), (4796, 32), (4796, 32))

In [231]:
def compute_accuracy(model,X,Y,M):
    prediction=model.eval_model(X)
    
    numerator=np.argmax(prediction,axis=-1)
    
    comprasion=np.sum(numerator==Y)
    
    denominator=np.sum(M)
    
    accuracy=comprasion/denominator
    return accuracy


In [232]:
print(f'Accuracy of Model is : {compute_accuracy(training,X,Y,M)}')

Accuracy of Model is : 0.9617193341255188


In [312]:
def pred(sentence,tokenize_sentence=tokenize_sentence):
    output=''
    tokenize_sentence=tokenize_sentence(sentence,vocab_word)
    tokenize_sentence=np.array(tokenize_sentence)
    tokenize_sentence=np.expand_dims(tokenize_sentence,axis=0)
    prediction=training.eval_model(tokenize_sentence)
    prediction=np.argmax(prediction,axis=-1)[0]
    for i in prediction:
        tag=number_to_tags[int(i)]
        output+=tag+' '
    return output


In [324]:
print(f'display actual output {tag[0]}')
print(f'display predicted output {pred(sentence[0])}')

display actual output  O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O
display predicted output O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O 
